<a href="https://colab.research.google.com/github/EbubeTheGoat/Asthma_medical_chatbot/blob/main/Medical_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3


In [ ]:
conn = sqlite3.connect('pharmacy.db')
print("Opened database successfully")

In [ ]:
conn.execute("""
CREATE TABLE disease(
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name TEXT,
  symptoms TEXT,
  treatment TEXT
)
 """
)

In [ ]:
conn.commit()

In [ ]:
conn.execute("""
INSERT INTO disease(name,symptoms,treatment)
VALUES(?,?,?)
""",(
    "Asthma",
    "Shortness of breath,chest tightness,wheezing,coughing(especially at night)",
    "Inhaled bronchodilators(e.g.,salbutamol),inhaled corticosteroid avoids triggers"
))
conn.commit()
print("Inserted successfully")

In [ ]:
cursor = conn.execute("SELECT * FROM disease")
for row in cursor:
    print(row)

In [ ]:
Model = "meta-llama/Llama-3.1-8B-Instruct"

In [ ]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
def get_disease_info(disease_name):
  conn = sqlite3.connect('pharmacy.db')
  cursor = conn.execute("""
  SELECT name, symptoms, treatment
  FROM disease
  WHERE name LIKE ?
  """,(f"%{disease_name}%",))
  row = cursor.fetchone()
  if row:
    return{
        "name": row[0],
        "symptoms":row[1],
        "treatment":row[2]
    }
  else:
    return None

In [ ]:
!pip install bitsandbytes accelerate

In [ ]:
from transformers import BitsAndBytesConfig

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)
print("Using 4-bit quantization")

In [ ]:
pip install --upgrade git+https://github.com/huggingface/transformers.git

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(Model,trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    Model,quantization_config = quant_config,device_map="auto",
    torch_dtype = torch.float16,trust_remote_code = True)

In [ ]:
def chatbot_response(message,history):
  try:
    disease = get_disease_info(message)

    if disease:
      system_context = (
          f"You are a medical assistant. Here is information from the database:\n\n"
          f"Disease: {disease['name']}\n"
          f"Symptoms: {disease['symptoms']}\n"
          f"Treatment: {disease['treatment']}\n\n"
          f"Answer the user's question based on the information provided."
          )
    else:
      system_context = (
          "You are a helpful medical assistant. Answer questions about health and diseases.\n\n "
      )
    conversation = system_context

    messages = [{"role":"system","content":system_context}]
    recent_history=history[-5:] if len(history) > 5 else history
    for user_msg,assistant_msg in recent_history:
      messages.append({"role":"user","content":user_msg})
      messages.append({"role":"assistant","content":assistant_msg})

    messages.append({"role":"user","content":message})
    inputs = tokenizer.apply_chat_template(messages,
                                           return_tensors = "pt",truncation = True,
                                           padding= True,add_generation_prompt=True,
                                           return_dict=True).to("cuda")

    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    print(f"Generating response.....")

    streamer = TextIteratorStreamer(tokenizer,timeout=None,skip_prompt=True,skip_special_tokens=True)

    generation_kwarg= dict(
        **inputs,
        streamer = streamer,
        max_new_tokens =1024,
        temperature = 0.7,
        do_sample = True,
        pad_token_id = tokenizer.eos_token_id,
        eos_token_id = tokenizer.eos_token_id,
        repetition_penalty = 1.1

   )
    thread = Thread(target=model.generate,kwargs=generation_kwarg)
    thread.start()
    partial_message = ""
    for new_token in streamer:
        if new_token:
          partial_message += new_token
          yield partial_message
  except Exception as e:
    error_msg=f"An error occurred: {str(e)}"
    print(error_msg)
    return f"Error details for debugging:{str(e)}"


In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread

In [ ]:
demo = gr.ChatInterface(
    chatbot_response,
    title = "Medical Assistant",
    description = "Ask about diseases,symptoms,or treatments.",
    examples=[
        "What causes asthma?",
        "Tell me about malaria symptoms"
    ],
    concurrency_limit=5
)

if __name__ == "__main__":
  print("Launching the demo...")
  demo.launch(auth=("pharm","doctor"),inbrowser = True,share= True)